## Notebook to test different codes and functions.

In [1]:
#folium is the library used to visualize maps
import folium
#library to call Open Route Service(ORS)'s client and requests
from openrouteservice import client
import pandas as pd 
import numpy as np 
import plotly as py
import json

#from isochrones import * 

#i like to be able to see all my columns in a pandas dataframe. this would allow it 
pd.set_option("display.max_columns", None)




In [2]:
from config import api_key
clnt = client.Client(key=api_key) 

In [28]:
#loading up train station data. This file contains the csv data for KL train stations. 
file_url = 'resources/data/raw_data/klang_valley_stations.csv'

#Stop ID is unique for every station. So that column will be used for index
test_data = pd.read_csv(file_url)

lines = list(test_data['Route Name'].unique())
maps = []
params_iso = {'profile': 'foot-walking', 
              'range': [900], # 900/60 = 15 mins
              'interval': 300,
              'attributes': ['area', 'reachfactor', 'total_pop'] # Get population count for isochrones
             }

In [4]:
lines

['Ampang Line',
 'Seremban Line',
 'Port Klang Line',
 'KLIA Ekspres',
 'Kelana Jaya Line',
 'Skypark Link',
 'KLIA Transit',
 'KL Monorail Line',
 'BRT Sunway Line',
 'MRT Sungai Buloh - Kajang Line',
 'Sri Petaling Line']

In [5]:
#def dictSetup(dataframe):
 #   station_dict = dataframe.to_dict(orient='index')
  #  for name, station in station_dict.items():
   #     station['locations'] = [station['Longitude'],station['Latitude']]
    #return station_dict

In [24]:
#Perform isochrone request and generates a new column in the stations dataframe containing isochrone data for that station.
#this will save the isochrones requested from Open Route Service(ORS)
#ORS will return the isochrones in geoJSON format
def isoGeoJsonRetriever(parameters,stations,client):

    #ORS isochrones API takes input list of coordinates in field called location. This line creates that column
    stations['locations']  = stations.apply(lambda row: list([row.loc["Longitude"],row.loc["Latitude"]]) , axis = 1)
    iso_list = []

    for index, row in stations.iterrows():
        print("Retrieving Isochrone of {} station".format(stations.loc[index,'Name']))
        parameters['locations'] = [row.loc['locations']]
        temp_iso = client.isochrones(**parameters)
        temp_iso = json.dumps(temp_iso)
        iso_list.append(temp_iso)
        print("Success")
    
    stations['iso'] = iso_list

    return

In [7]:
def isoVisualizer(maps,stations, map_icon = ""):
    style_function = lambda x: {'color': '#4ef500' if x['properties']['value']<400 else ('#2100f5' if x['properties']['value']<700.0 else '#f50000'),
                                'fillOpacity' : 0.35 if x['properties']['value']<400 else (0.25 if 400.0<x['properties']['value']<700.0 else 0.05),
                                'weight':2,
                                'fillColor' :'#4ef500' if x['properties']['value']<400 else ('#2100f5' if 400.0<x['properties']['value']<700.0 else '#f50000')}
                                                    #('#6234eb' if x['properties']['value']==600.0 else '#6234eb')
    stations['locations']  = stations.apply(lambda row: list([row.loc["Longitude"],row.loc["Latitude"]]) , axis = 1)

    for index, row in stations.iterrows():
        folium.features.GeoJson(row['iso'],style_function = style_function).add_to(maps) # Add GeoJson to map
        if map_icon!="":
            folium.map.Marker(list(reversed(row['locations'])), # reverse coords due to weird folium lat/lon syntax
                                icon=folium.Icon(color='lightgray',
                                            icon_color='#cc0000',
                                            icon=map_icon,
                                            prefix='fa',
                                                ),
                            popup=row['Name'],
                            ).add_to(maps) # Add apartment locations to map      
    print("Done!")

In [8]:
def toMapORS(data,line,params_iso,client):
    # Set up folium map
    if not line in data.values:
        print('{} is not in data frame'.format(line))
        temp = data['Route Name'].unique()
        print('Choose from the following: ')
        print(temp)
        return
    if line != None:
        data = data[data['Route Name']==line]
    starting_location = (data['Latitude'].iloc[0],data['Longitude'].iloc[0])
    mapped = folium.Map(tiles='OpenStreetMap', location=starting_location, zoom_start=11)

    isoGeoJsonRetriever(params_iso,data,client)
    isoVisualizer(mapped,data)
    return mapped,data

In [9]:
#method that uses input dataframe with iso. Returns Map of isochrones.
def isoMapper(data,icon = 'train'):
    ## Set up folium map
    starting_location = (data['Latitude'].iloc[0],data['Longitude'].iloc[0])
    mapped = folium.Map(tiles='OpenStreetMap', location=starting_location, zoom_start=13)

    ##Add legend to Map
    #colormap = linear.RdYlBu_08.scale(station_stats[field_to_color_by].quantile(0.05),
                                     # station_stats[field_to_color_by].quantile(0.95))

    #converts Dataframe to Dictionary, necessary for the subsequent functions
    isoVisualizer(mapped,data,icon)

    return mapped

In [10]:
data_all = pd.read_csv('resources/data/klang_valley_stations_isochrones.csv')

selected_data = data_all[data_all['City']=='Kuala Lumpur']

selected_data = selected_data[(selected_data['Route Name'].isin(['Ampang Line']))]

selected_map = isoMapper(selected_data)


Done!


In [19]:
print(selected_data['iso'][0])

{"type": "FeatureCollection", "bbox": [101.687526, 3.176238, 101.701856, 3.19394], "features": [{"type": "Feature", "properties": {"group_index": 0, "value": 300.0, "center": [101.69540263355309, 3.1858327450971533], "area": 159328.79, "reachfactor": 0.2921, "total_pop": 847.0}, "geometry": {"coordinates": [[[101.694422, 3.182262], [101.694768, 3.182163], [101.695253, 3.182599], [101.69655, 3.183927], [101.697077, 3.185471], [101.697091, 3.188183], [101.695523, 3.18929], [101.694812, 3.189394], [101.694543, 3.189372], [101.694573, 3.189013], [101.695008, 3.184095], [101.694663, 3.18299], [101.694556, 3.182731], [101.694489, 3.182496], [101.694422, 3.182262]]], "type": "Polygon"}}, {"type": "Feature", "properties": {"group_index": 0, "value": 600.0, "center": [101.69540263355309, 3.1858327450971533], "area": 980240.62, "reachfactor": 0.4493, "total_pop": 7814.0}, "geometry": {"coordinates": [[[101.691158, 3.181704], [101.691202, 3.181347], [101.692238, 3.180321], [101.693147, 3.179841],

In [18]:
selected_map

In [34]:
selected_data['Longitude'][0]

3.185821

In [47]:
temp_para

{'profile': 'foot-walking',
 'range': [900],
 'interval': 300,
 'attributes': ['area', 'reachfactor', 'total_pop'],
 'locations': [101.695335, 3.185821]}

In [49]:
temp_para = params_iso.copy()
coords = [[selected_data['Longitude'][0],selected_data['Latitude'][0]]]
temp_para["locations"] = coords

temp_iso = clnt.isochrones(**temp_para)

In [62]:
type(temp_iso)
json.dumps(temp_iso)

'{"type": "FeatureCollection", "bbox": [101.687526, 3.176238, 101.701856, 3.19394], "features": [{"type": "Feature", "properties": {"group_index": 0, "value": 300.0, "center": [101.69540263355309, 3.1858327450971533], "area": 159328.79, "reachfactor": 0.2921, "total_pop": 847.0}, "geometry": {"coordinates": [[[101.694422, 3.182262], [101.694768, 3.182163], [101.695253, 3.182599], [101.69655, 3.183927], [101.697077, 3.185471], [101.697091, 3.188183], [101.695523, 3.18929], [101.694812, 3.189394], [101.694543, 3.189372], [101.694573, 3.189013], [101.695008, 3.184095], [101.694663, 3.18299], [101.694556, 3.182731], [101.694489, 3.182496], [101.694422, 3.182262]]], "type": "Polygon"}}, {"type": "Feature", "properties": {"group_index": 0, "value": 600.0, "center": [101.69540263355309, 3.1858327450971533], "area": 980240.62, "reachfactor": 0.4493, "total_pop": 7814.0}, "geometry": {"coordinates": [[[101.691158, 3.181704], [101.691202, 3.181347], [101.692238, 3.180321], [101.693147, 3.179841]

In [58]:
location_data  = json.dumps(selected_data['iso'][0])
type(json.dumps(location_data))

str

In [30]:
temp_style_function = lambda x: {'color': '#4ef500' if x['properties']['value']<400 else ('#2100f5' if x['properties']['value']<700.0 else '#f50000'),
                                'fillOpacity' : 0.35 if x['properties']['value']<400 else (0.25 if 400.0<x['properties']['value']<700.0 else 0.05),
                                'weight':2,
                                'fillColor' :'#4ef500' if x['properties']['value']<400 else ('#2100f5' if 400.0<x['properties']['value']<700.0 else '#f50000')}
                                                    #('#6234eb' if x['properties']['value']==600.0 else '#6234eb')
location_data  = json.dumps(selected_data['iso'][0])
location_data = location_data.replace("'", '"')
temp_mapped = folium.Map(tiles='OpenStreetMap', zoom_start=13)
folium.features.GeoJson(location_data,style_function = temp_style_function).add_to(temp_mapped) # Add GeoJson to map



OSError: [Errno 36] File name too long: '"{"type": "FeatureCollection", "bbox": [101.687526, 3.176238, 101.701856, 3.19394], "features": [{"type": "Feature", "properties": {"group_index": 0, "value": 300.0, "center": [101.69540263355309, 3.1858327450971533], "area": 159328.79, "reachfactor": 0.2921, "total_pop": 847.0}, "geometry": {"coordinates": [[[101.694422, 3.182262], [101.694768, 3.182163], [101.695253, 3.182599], [101.69655, 3.183927], [101.697077, 3.185471], [101.697091, 3.188183], [101.695523, 3.18929], [101.694812, 3.189394], [101.694543, 3.189372], [101.694573, 3.189013], [101.695008, 3.184095], [101.694663, 3.18299], [101.694556, 3.182731], [101.694489, 3.182496], [101.694422, 3.182262]]], "type": "Polygon"}, "id": "0"}, {"type": "Feature", "properties": {"group_index": 0, "value": 600.0, "center": [101.69540263355309, 3.1858327450971533], "area": 980199.44, "reachfactor": 0.4493, "total_pop": 7814.0}, "geometry": {"coordinates": [[[101.691158, 3.181704], [101.691202, 3.181

In [23]:
test_data

,Name,Stop ID,Service Provider Name,Latitude,Longitude,ROUTE ID,Route Name,Line Number,Line Colour,Colour Hex Code,City
0,SENTUL TIMUR,AG1,Rapid KL,3.185821,101.695335,AG,Ampang Line,3,Orange,#fe8e10,Kuala Lumpur
1,PUDU,AG10,Rapid KL,3.134920,101.712011,AG,Ampang Line,3,Orange,#fe8e10,Kuala Lumpur
2,CHAN SOW LIN,AG11,Rapid KL,3.128025,101.715546,AG,Ampang Line,3,Orange,#fe8e10,Kuala Lumpur
3,MIHARJA,AG12,Rapid KL,3.120931,101.717861,AG,Ampang Line,3,Orange,#fe8e10,Kuala Lumpur
4,MALURI,AG13,Rapid KL,3.123276,101.726946,AG,Ampang Line,3,Orange,#fe8e10,Kuala Lumpur
...,...,...,...,...,...,...,...,...,...,...,...
200,SULTAN ISMAIL,SP5,Rapid KL,3.161185,101.694127,SP,Sri Petaling Line,4,Maroon,#721422,Kuala Lumpur
201,BANDARAYA,SP6,Rapid KL,3.155548,101.694406,SP,Sri Petaling Line,4,Maroon,#721422,Kuala Lumpur
202,MASJID JAMEK,SP7,Rapid KL,3.149462,101.696396,SP,Sri Petaling Line,4,Maroon,#721422,Kuala Lumpur
203,PLAZA RAKYAT,SP8,Rapid KL,3.144264,101.702054,SP,Sri Petaling Line,4,Maroon,#721422,Kuala Lumpur


In [29]:

# I should edit the toMap function, if params_iso or client is not passed then just map without calling isochrones from ORS         
selected_lines = lines[0:1]
  
for line in selected_lines:
    maps.append(toMapORS(test_data,line,params_iso,clnt))

/tmp/ipykernel_5956/753795925.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['locations']  = stations.apply(lambda row: list([row.loc["Longitude"],row.loc["Latitude"]]) , axis = 1)


Retrieving Isochrone of SENTUL TIMUR station
Success
Retrieving Isochrone of PUDU station
Success
Retrieving Isochrone of CHAN SOW LIN station
Success
Retrieving Isochrone of MIHARJA station
Success
Retrieving Isochrone of MALURI station
Success
Retrieving Isochrone of PANDAN JAYA station
Success
Retrieving Isochrone of PANDAN INDAH station
Success
Retrieving Isochrone of CEMPAKA station
Success
Retrieving Isochrone of CAHAYA station
Success
Retrieving Isochrone of AMPANG station
Success
Retrieving Isochrone of SENTUL station
Success
Retrieving Isochrone of TITIWANGSA station
Success
Retrieving Isochrone of PWTC station
Success
Retrieving Isochrone of SULTAN ISMAIL station


/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/home/nabilersyad/miniconda3/envs/ors/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,


Success
Retrieving Isochrone of BANDARAYA station
Success
Retrieving Isochrone of MASJID JAMEK station
Success
Retrieving Isochrone of PLAZA RAKYAT station
Success
Retrieving Isochrone of HANG TUAH station
Success
Done!


/tmp/ipykernel_5956/753795925.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['iso'] = iso_list


In [34]:
maps[0][0]

In [19]:
maps[8][1]

,Name,Stop ID,Service Provider Name,Latitude,Longitude,ROUTE ID,Route Name,Line Number,Line Colour,Colour Hex Code,City,locations,iso
138,SUNWAY-SETIA JAYA,SB1,Rapid Bus,3.08292,101.61228,SB,BRT Sunway Line,B1,Dark Green,#1E4D2B,Kuala Lumpur,"[101.61228, 3.08292]","{'type': 'FeatureCollection', 'bbox': [101.602..."
139,MENTARI,SB2,Rapid Bus,3.07613,101.61023,SB,BRT Sunway Line,B1,Dark Green,#1E4D2B,Kuala Lumpur,"[101.61023, 3.07613]","{'type': 'FeatureCollection', 'bbox': [101.600..."
140,SUNWAY LAGOON,SB3,Rapid Bus,3.07065,101.61079,SB,BRT Sunway Line,B1,Dark Green,#1E4D2B,Kuala Lumpur,"[101.61079000000001, 3.07065]","{'type': 'FeatureCollection', 'bbox': [101.602..."
141,SUNMED,SB4,Rapid Bus,3.06554,101.60861,SB,BRT Sunway Line,B1,Dark Green,#1E4D2B,Kuala Lumpur,"[101.60861, 3.06554]","{'type': 'FeatureCollection', 'bbox': [101.596..."
142,SUNU MONASH,SB5,Rapid Bus,3.06530,101.60161,SB,BRT Sunway Line,B1,Dark Green,#1E4D2B,Kuala Lumpur,"[101.60161, 3.0653]","{'type': 'FeatureCollection', 'bbox': [101.592..."
143,SOUTH QUAY-USJ1,SB6,Rapid Bus,3.06162,101.59681,SB,BRT Sunway Line,B1,Dark Green,#1E4D2B,Kuala Lumpur,"[101.59680999999999, 3.06162]","{'type': 'FeatureCollection', 'bbox': [101.589..."
144,USJ7,SB7,Rapid Bus,3.05525,101.59198,SB,BRT Sunway Line,B1,Dark Green,#1E4D2B,Kuala Lumpur,"[101.59198, 3.05525]","{'type': 'FeatureCollection', 'bbox': [101.581..."


In [18]:
location = [data_kl.loc[0]['Longitude'],data_kl.loc[0]['Latitude']]

In [27]:
location

[101.67218220000001, 3.2145439]

In [24]:
isochrone = clnt.isochrones(**params_iso)

In [31]:
json_object = json.dumps(isochrone) 
print(json_object)

{"type": "FeatureCollection", "bbox": [101.666837, 3.2043, 101.681121, 3.225201], "features": [{"type": "Feature", "properties": {"group_index": 0, "value": 300.0, "center": [101.67217506885774, 3.2144347416450167], "area": 325029.53, "reachfactor": 0.5959, "total_pop": 2980.0}, "geometry": {"coordinates": [[[101.669504, 3.213705], [101.66951, 3.213641], [101.670229, 3.212827], [101.671256, 3.211815], [101.672271, 3.210862], [101.672347, 3.210815], [101.672707, 3.210799], [101.674912, 3.213113], [101.675311, 3.213614], [101.675308, 3.213974], [101.672953, 3.218065], [101.672593, 3.21808], [101.671968, 3.217945], [101.671879, 3.217919], [101.671632, 3.217816], [101.670963, 3.217122], [101.670275, 3.216101], [101.669784, 3.215236], [101.669508, 3.214001], [101.669504, 3.213705]]], "type": "Polygon"}}, {"type": "Feature", "properties": {"group_index": 0, "value": 600.0, "center": [101.67217506885774, 3.2144347416450167], "area": 1219315.23, "reachfactor": 0.5589, "total_pop": 7370.0}, "ge

In [5]:
station_kl_dict = dictSetup(data_kl)


In [6]:
location = (station_kl_dict[1]['Latitude'],station_kl_dict[1]['Longitude'])

In [7]:
mapped = folium.Map(tiles='OpenStreetMap',location=location ,zoom_start=11)




In [8]:
x = station_kl_dict[1]['iso']
x = x.replace("'", '"')
j = json.loads(x)

In [9]:
from isochrones import * 

isoVisualizer(mapped,station_kl_dict)

Done!


In [11]:
mapzzz =isoMapper(data_kl)
mapzzz

Done!


In [12]:
seremban_line_stations = list(data_kl[data_kl['Route Name']=='Seremban Line'].index)

In [14]:
seremban_line_stations_dict = stationSubset(station_kl_dict,seremban_line_stations)

In [16]:
seremban_line_stations_df = data_kl[data_kl['Route Name']=='Seremban Line']
seremban_map = isoMapper(seremban_line_stations_df)
seremban_map


Done!


In [24]:
def isoVisualizerTest(maps,stations, map_icon = 'train'):
    style_function = lambda x: {'color': '#4ef500' if x['properties']['value']<400 else ('#2100f5' if x['properties']['value']<700.0 else '#f50000'),
                                'fillOpacity' : 0.35 if x['properties']['value']<400 else (0.25 if 400.0<x['properties']['value']<700.0 else 0.05),
                                'weight':2,
                                'fillColor' :'#4ef500' if x['properties']['value']<400 else ('#2100f5' if 400.0<x['properties']['value']<700.0 else '#f50000')}
                                                    #('#6234eb' if x['properties']['value']==600.0 else '#6234eb')
    
    for name, station in stations.items():
        station_iso_temp = station['iso']
        station_iso_temp = station_iso_temp.replace("'", '"')
        folium.features.GeoJson(station_iso_temp,style_function = style_function).add_to(maps) # Add GeoJson to map
        if map_icon!="":
            folium.map.Marker(list(reversed(station['locations'])), # reverse coords due to weird folium lat/lon syntax
                                icon=folium.Icon(color='lightgray',
                                            icon_color='#cc0000',
                                            icon=map_icon,
                                            prefix='fa',
                                                ),
                            popup=station['Name'],
                            ).add_to(maps) # Add apartment locations to map
            
    print("Done!")

In [14]:
mapped_ampang = folium.Map(tiles='OpenStreetMap',location=location ,zoom_start=11)
isoVisualizer(mapped_ampang,ampang_line_stations_dict,)

Done!


In [15]:
mapped_ampang

In [38]:
import os
import time
from selenium import webdriver
delay=10
 
#Save the map as an HTML file
fn='testmap.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
mapped_ampang.save(fn)
 
#Open a browser window...
browser = webdriver.Firefox()
#..that displays the map...
browser.get(tmpurl)
#Give the map tiles some time to load
time.sleep(delay)
#Grab the screenshot
browser.save_screenshot('map.png')
#Close the browser
browser.quit()

In [11]:
help(mapped,station_kl_dict)

Help on function isoVisualizer in module isochrones:

isoVisualizer(maps, stations)
    #input a Folium Map and Stations dictionary containing ISO data. this will draw the ISO lines on the folium map object



In [10]:
mapped

In [15]:
eval(station_kl_dict[200]['iso']) 

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'group_index': 0,
    'value': 300.0,
    'center': [101.60161015012037, 3.0652656055878724],
    'area': 324775.13,
    'reachfactor': 0.5955,
    'total_pop': 1338.0},
   'geometry': {'coordinates': [[[101.597829, 3.065154],
      [101.598128, 3.064722],
      [101.600167, 3.062499],
      [101.60115, 3.06212],
      [101.60151, 3.062118],
      [101.605368, 3.065222],
      [101.605349, 3.065581],
      [101.605265, 3.06568],
      [101.603415, 3.067457],
      [101.602689, 3.068106],
      [101.602515, 3.068166],
      [101.602195, 3.068175],
      [101.59784, 3.065514],
      [101.597829, 3.065154]]],
    'type': 'Polygon'},
   'id': '0'},
  {'type': 'Feature',
   'properties': {'group_index': 0,
    'value': 600.0,
    'center': [101.60161015012037, 3.0652656055878724],
    'area': 1189454.82,
    'reachfactor': 0.5452,
    'total_pop': 6902.0},
   'geometry': {'coordinates': [[[101.594105, 3.065529

In [40]:
style_function = lambda x: {'color': '#4ef500' if x['properties']['value']<400 else ('#2100f5' if x['properties']['value']<700.0 else '#f50000'),
                                'fillOpacity' :0.25,
                                'weight':2,
                                'fillColor' :'#92daf0'}

temp = eval(station_kl_dict[200]['iso']) 
print(temp)
folium.features.GeoJson(temp,style_function = style_function).add_to(mapped) # Add GeoJson to map
mapped

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'group_index': 0, 'value': 300.0, 'center': [101.60161015012037, 3.0652656055878724], 'area': 324775.13, 'reachfactor': 0.5955, 'total_pop': 1338.0}, 'geometry': {'coordinates': [[[101.597829, 3.065154], [101.598128, 3.064722], [101.600167, 3.062499], [101.60115, 3.06212], [101.60151, 3.062118], [101.605368, 3.065222], [101.605349, 3.065581], [101.605265, 3.06568], [101.603415, 3.067457], [101.602689, 3.068106], [101.602515, 3.068166], [101.602195, 3.068175], [101.59784, 3.065514], [101.597829, 3.065154]]], 'type': 'Polygon'}, 'id': '0'}, {'type': 'Feature', 'properties': {'group_index': 0, 'value': 600.0, 'center': [101.60161015012037, 3.0652656055878724], 'area': 1189454.82, 'reachfactor': 0.5452, 'total_pop': 6902.0}, 'geometry': {'coordinates': [[[101.594105, 3.065529], [101.594111, 3.065169], [101.594147, 3.065114], [101.594671, 3.064349], [101.596278, 3.063028], [101.600531, 3.061459], [101.602098, 3.06

In [28]:
station_kl_dict[1]['iso']

"{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'group_index': 0, 'value': 300.0, 'center': [101.67552795280952, 3.2045679883189795], 'area': 153781.46, 'reachfactor': 0.282, 'total_pop': 1021.0}, 'geometry': {'coordinates': [[[101.672769, 3.205265], [101.672785, 3.205129], [101.673509, 3.202695], [101.674189, 3.202056], [101.674543, 3.20199], [101.676433, 3.201641], [101.676729, 3.202021], [101.676616, 3.203818], [101.674158, 3.206536], [101.673957, 3.206774], [101.673682, 3.206542], [101.672939, 3.205583], [101.672769, 3.205265]]], 'type': 'Polygon'}, 'id': '0'}, {'type': 'Feature', 'properties': {'group_index': 0, 'value': 600.0, 'center': [101.67552795280952, 3.2045679883189795], 'area': 923224.06, 'reachfactor': 0.4232, 'total_pop': 7403.0}, 'geometry': {'coordinates': [[[101.669128, 3.205928], [101.669218, 3.205579], [101.670317, 3.201664], [101.672031, 3.200279], [101.675519, 3.198624], [101.675861, 3.198513], [101.678235, 3.198579], [101.678548, 3

In [29]:

params_iso = {'profile': 'foot-walking', 
              'range': [900], # 900/60 = 15 mins
              'interval': 300,
              'attributes': ['area', 'reachfactor', 'total_pop'] # Get population count for isochrones
             }
params_iso['locations'] = [station_kl_dict[1]['locations']]
isos = clnt.isochrones(**params_iso)

In [30]:
map_sg = folium.Map(tiles='OpenStreetMap',location=location ,zoom_start=11)


In [11]:
station_sg_dict = dictSetup(data_sg)
for name, station in station_sg_dict.items():
        folium.map.Marker(list(reversed(station['locations'])), # reverse coords due to weird folium lat/lon syntax
                            icon=folium.Icon(color='lightgray',
                                        icon_color='#cc0000',
                                        icon='train',
                                        prefix='fa',
                                            ),
                            popup=station['Name'],
                            ).add_to(map_sg) # Add apartment locations to map

In [12]:
map_sg